### Finding best Minimally Complex model for supreme court voting dataset using integer representation

In [1]:
# Imports
import sys
sys.path.insert(0, '../')
from python.mcm_binary import mcm

In [2]:
model = mcm('../data/SC_voting/US_SupremeCourt_n9_N895.txt')

In [3]:
# Evidence of IM with original basis (Should be -5258)
print('Independent model in the original basis')
print('Evidence: ', model.calc_log_evidence(model.mcms[-1]))

Independent model in the original basis
Evidence:  -5258.100240438084


In [4]:
# Finding best MCM in original basis (Should be [[0, 2, 3, 4, 6], [1, 5, 7, 8]] with evidence -3300.4)
model.find_best_mcm()
print('Best MCM in original basis (Exhaustive search)')
print('MCM: ', model.best_mcm)
print('Evidence: ', model.best_evidence)

Best MCM in original basis (Exhaustive search)
MCM:  [[0, 2, 3, 4, 6], [1, 5, 7, 8]]
Evidence:  -3300.395469673639


In [5]:
# Finding best MCM in original basis using a Greedy search (Should be [[0, 2, 3, 4, 6], [1, 5, 7, 8]] with evidence -3300.4)
model.find_best_mcm(method='greedy')
print('Best MCM in original basis (Greedy search)')
print('MCM: ', model.best_mcm)
print('Evidence: ', model.best_evidence)

Best MCM in original basis (Greedy search)
MCM:  [[0, 4, 2, 3, 6], [1, 5, 7, 8]]
Evidence:  -3300.395469673639


In [20]:
# Finding best MCM in original basis using the divide and conquer scheme
model.find_best_mcm(method='divide_and_conquer', print_search=True)
print('Best MCM in original basis (Divide and conquer)')
print('MCM: ', model.best_mcm)
print('Evidence: ', model.best_evidence)

Start splitting procedure:  [[0, 1, 2, 3, 4, 5, 6, 7, 8]] -3305.54657591369
[[1, 2, 3, 4, 5, 6, 7, 8], [0]] -3506.473953479855
New intermediate best:  [[1, 2, 3, 4, 5, 6, 7, 8], [0]] -3506.473953479855
[[0, 2, 3, 4, 5, 6, 7, 8], [1]] -3423.151089359927
New intermediate best:  [[0, 2, 3, 4, 5, 6, 7, 8], [1]] -3423.151089359927
[[0, 1, 3, 4, 5, 6, 7, 8], [2]] -3505.3254342779846
[[0, 1, 2, 4, 5, 6, 7, 8], [3]] -3515.4197429054993
[[0, 1, 2, 3, 5, 6, 7, 8], [4]] -3500.4252510768592
[[0, 1, 2, 3, 4, 6, 7, 8], [5]] -3553.896470753466
[[0, 1, 2, 3, 4, 5, 7, 8], [6]] -3503.42222728104
[[0, 1, 2, 3, 4, 5, 6, 8], [7]] -3567.686532154147
[[0, 1, 2, 3, 4, 5, 6, 7], [8]] -3539.81806525089
[[2, 3, 4, 5, 6, 7, 8], [1, 0]] -3638.0787001949993
New intermediate best:  [[2, 3, 4, 5, 6, 7, 8], [1, 0]] -3638.0787001949993
[[0, 3, 4, 5, 6, 7, 8], [1, 2]] -3599.3100345335324
New intermediate best:  [[0, 3, 4, 5, 6, 7, 8], [1, 2]] -3599.3100345335324
[[0, 2, 4, 5, 6, 7, 8], [1, 3]] -3654.4803774308366
[[0, 2

In [6]:
# Finding the best IM (Should be [72, 160, 384, 17, 65, 5, 130, 260, 64] with evidence -3327)
model.find_best_im(max_interactions=2)
print('Best Independent model')
print('IM: ', model.best_im)
print('Evidence: ', model.calc_log_evidence(model.mcms[-1]))

Best Independent model
IM:  [ 72 160 384  17  65   5 130 260  64]
Evidence:  -3327.079945846724


In [7]:
# Finding best MCM in optimal basis (Should be [[0], [1, 2, 6], [3, 4, 5, 7, 8]] with evidence -3154.4)
model.find_best_mcm()
print('Best MCM in the optimal basis (Exhaustive search)')
print('MCM: ', model.best_mcm)
print('Evidence: ', model.best_evidence)

Best MCM in the optimal basis (Exhaustive search)
MCM:  [[0], [1, 2, 6], [3, 4, 5, 7, 8]]
Evidence:  -3154.421230299754


In [8]:
# Finding the best MCM in the optimal basis using Greedy search ([[0, 4, 3, 5], [1, 2, 6, 7, 8]] -3163.252527450342)
model.find_best_mcm(method='greedy')
print('Best MCM in the optimal basis (Greedy search)')
print('MCM: ', model.best_mcm)
print('Evidence: ', model.best_evidence)

Best MCM in the optimal basis (Greedy search)
MCM:  [[0, 4, 3, 5], [1, 2, 6, 7, 8]]
Evidence:  -3163.252527450342


In [9]:
# Finding the best MCM in the optimal basis using the divide and conquer scheme
model.find_best_mcm(method='divide_and_conquer')
print('Best MCM in the optimal basis (Divide and conquer)')
print('MCM: ', model.best_mcm)
print('Evidence: ', model.best_evidence)

Best MCM in the optimal basis (Divide and conquer)
MCM:  [[1, 2, 6], [3, 5, 4, 7, 8], [0]]
Evidence:  -3154.421230299755
